<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork900-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Historical Stock Price and Revenue Analysis — Tesla & GameStop

**Short description:**  
This notebook downloads historical stock price data (via `yfinance`) and quarterly revenue tables (from provided HTML pages), cleans the data, and visualizes price vs revenue for Tesla and GameStop.

**Objectives**
- Extract and clean historical stock prices.
- Extract and clean revenue tables from HTML sources.
- Compare price vs. revenue graphs.

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & instructions  
2. Functions and helper definitions  
3. Tesla — price & revenue extraction and cleaning  
4. GameStop — price & revenue extraction and cleaning  
5. Visualization: price vs revenue plots 


## 1) Dependencies & execution instructions

**Recommended local execution steps:**:
1. Create and activate a virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate`  (Linux / macOS) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Launch Jupyter and open this notebook:
   - `jupyter notebook`

The notebook requires internet access to fetch data from `yfinance` and the referenced HTML pages.


In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## 2) Functions and helper definitions

This section contains the helper function for graphing stock data vs revenue data for a given stock.

In [2]:
def make_graph(stock_data, revenue_data, stock):
    """
    Plot historical share price (top) and historical revenue (bottom) for the given stock.

    Parameters
    ----------
    stock_data : DataFrame
        Must contain a 'Date' column and a 'Close' column (closing price).
    revenue_data : DataFrame
        Must contain a 'Date' column and a 'Revenue' column (numeric or numeric strings).
    stock : str
        Title for the plot (e.g., 'Tesla').

    """
    # Create subplots: price on top, revenue below
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Historical Share Price", "Historical Revenue"),
                        vertical_spacing=0.3)
    
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']

    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    fig.update_layout(showlegend=False,
                      height=900,
                      title=stock,
                      xaxis_rangeslider_visible=True)
    fig.show()

## 3) Tesla: download and clean price & revenue

This section downloads Tesla stock price via `yfinance` and reads the Tesla revenue table from the provided HTML URL. It performs basic cleaning of the revenue column (removing `$` and commas, converting to numeric) and ensures dates are in datetime format.

In [3]:
# Download Tesla ticker data using yfinance
tsla = yf.Ticker("TSLA")
tesla_data = tsla.history(period="max")
tesla_data.reset_index(inplace=True)

tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [4]:
# Read Tesla revenue table from provided URL (course resource)
tesla_revenue_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(tesla_revenue_url).text

# Parse the page
soup = BeautifulSoup(html_data, "html.parser")

# Use pandas.read_html to extract tables
tesla_revenue = pd.read_html(tesla_revenue_url)[1]
tesla_revenue.columns = ['Date', 'Revenue']

# Clean the Revenue column: remove $ and commas and convert to numeric
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].astype(str).str.replace(r'[\$,]', "", regex=True)
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce')

# Drop missing/empty revenues and reset index
tesla_revenue.dropna(subset=['Revenue'], inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.reset_index(drop=True, inplace=True)

tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31.0
49,2010-06-30,28.0
50,2010-03-31,21.0
51,2009-09-30,46.0
52,2009-06-30,27.0


## 4) GameStop: download and clean price & revenue

This section downloads GME stock price via `yfinance` and extracts revenue from the provided HTML page. Similar cleaning steps are applied.

In [5]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693349,1.603295,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [6]:
gme_revenue_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(gme_revenue_url).text
soup = BeautifulSoup(html_data, "html.parser")

# List to store rows
rows = []

for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text.strip()
    revenue = col[1].text.strip()
    # Append row to list of rows
    rows.append({"Date": date, "Revenue": revenue})

# Create DataFrame
gme_revenue = pd.DataFrame(rows)

# Clean revenue: remove $ and commas and convert to numeric
gme_revenue["Revenue"] = gme_revenue['Revenue'].astype(str).str.replace(r'[\$,]', "", regex=True)
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors='coerce')

gme_revenue.dropna(subset=['Revenue'], inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue.reset_index(drop=True, inplace=True)

gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## 5) Visualization: price vs revenue

This section produces stacked/paired figures (price on top, revenue below) using Plotly for interactive inspection.

In [7]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [8]:
make_graph(gme_data, gme_revenue, 'GameStop')

<h2>Authors:</h2> 

Joseph Santarcangelo

Azim Hirjani


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2022-02-28        | 1.2     | Lakshmi Holla | Changed the URL of GameStop |
| 2020-11-10        | 1.1     | Malika Singla | Deleted the Optional part |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab       |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
